# Pumpen Grafiken
Hallo Marcel. 

Ich habe festgestellt, dass es am einfachsten ist, die COP-Werte zu visualisieren und zu verstehen, wenn man eine interaktive Grafik erstellt. Diese Grafiken werden neu gezeichnet, wenn man die Regler für verschiedene Werte bewegt. Alles in einer einzigen Grafik (oder in mehreren Grafiken) darzustellen, wäre sowohl in Bezug auf die Visualisierung als auch in Bezug auf den dafür benötigten Code ein unübersichtliches Durcheinander geworden. 

Idealerweise könnte ich dir die benötigten Dateien zur Verfügung stellen und du könntest sie selbst auf deinem Computer ausführen, aber du kannst auch einfach diese Website benutzen, um das von mir erstellte interaktive Dokument auszuführen, da du wahrscheinlich nicht mehrere andere Dinge installieren musst (oder willst). Der einzige Nachteil des Servers ist, dass die Erstellung einiger Grafiken ein wenig länger dauern könnte.

Wenn du noch weitere Fragen hast, kannst du mich gerne kontaktieren.

## Zum Ausführen benötigter Code (du kannst das ignorieren und überspringen)

In [27]:
from matplotlib import pyplot as plt
from ipywidgets import interact
import matplotlib
import numpy as np
import pandas as pd

COL_NAMES = [str(speed) + "%" for speed in range(0, 105, 5)]
COL_NAMES.insert(0, "Temperatur")

SHEET_NAMES = [
    "Vorlauftemperatur_30",
    "Vorlauftemperatur_35",
    "Vorlauftemperatur_40",
    "Vorlauftemperatur_45",
    "Vorlauftemperatur_50",
    "Vorlauftemperatur_55",
]  # Vorlauftemperatur should end at 55°C


def loadData(excel_file_path, sheet=0) -> pd.core.frame.DataFrame:
    cop = pd.read_excel(
        excel_file_path,
        sheet_name=sheet,
        names=COL_NAMES,
        skiprows=1,
    )
    MAX_AUSSENTEMP = 41  # HACK: Außentemperatur over 20°C is irrelevant
    return cop.iloc[:MAX_AUSSENTEMP]


def plotPumps(inverter_speed, vorlauftemperatur):
    # convert ints from the slider to strings
    inverter_speed = str(inverter_speed) + "%"
    vorlauftemperatur = "Vorlauftemperatur_" + str(vorlauftemperatur)

    cop20 = loadData("WPL 20 COP.xlsx", vorlauftemperatur)
    cop25 = loadData("WPL 25A COP.xlsx", vorlauftemperatur)
    cop07 = loadData("WPL 07HK Premium COP.xlsx", vorlauftemperatur)

    plt.figure(dpi=150)
    plt.plot(cop20["Temperatur"], cop20[inverter_speed], "^", label="WPL 20")
    plt.plot(cop25["Temperatur"], cop25[inverter_speed], "*", label="WPL 25A")
    plt.plot(cop07["Temperatur"], cop07[inverter_speed], "P", label="WPL 07HK Premium")
    plt.xticks(range(0, len(cop20["Temperatur"]), 4))  # grid with every 4th value
    plt.xlabel("Außentemperatur [°C]")
    plt.ylabel("COP")
    plt.grid(True, which="major", linestyle="-")
    plt.grid(True, which="minor", linestyle="--")
    plt.minorticks_on
    plt.title(
        f"COP mit {inverter_speed} Inverter-speed bei {vorlauftemperatur[-2:]}°C Vorlauftemperatur"
    )
    plt.legend()
    plt.show()


def heatmap(
    data, row_labels, col_labels, ax=None, cbar_kw=None, cbarlabel="", **kwargs
):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    -----------
    Code taken from https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap

    im = ax.imshow(data, cmap="RdYlGn", **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, shrink=True, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right", rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
    ax.grid(which="minor", color="w", linestyle="-", linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(
    im,
    data=None,
    valfmt="{x:.2f}",
    textcolors=("black", "white"),
    threshold=None,
    **textkw,
):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    -----------
    Code taken from https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max()) / 2.0

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center", verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts


def findPercentage(values: np.ndarray) -> np.ndarray:
    highest = max(values)
    percentages = np.array([])
    for v in values:
        percentages = np.append(percentages, 100 * v / highest)

    return percentages


def convertToPercentages(data: np.ndarray) -> np.ndarray:
    data = data.copy()
    data = data.T
    for i, row in enumerate(data):
        data[i] = findPercentage(row)

    return data.T


def plotHeatmaps(inverter_speed, vorlauftemperatur):
    inverter_speed = str(inverter_speed) + "%"
    vorlauftemperatur = "Vorlauftemperatur_" + str(vorlauftemperatur)

    cop20 = loadData("WPL 20 COP.xlsx", vorlauftemperatur)
    cop25 = loadData("WPL 25A COP.xlsx", vorlauftemperatur)
    cop07 = loadData("WPL 07HK Premium COP.xlsx", vorlauftemperatur)
    cops = np.array(
        [cop20[inverter_speed], cop25[inverter_speed], cop07[inverter_speed]]
    )
    percentages = convertToPercentages(cops)

    fig, ax = plt.subplots(figsize=(25, 8), dpi=150)
    im, _ = heatmap(
        percentages,
        ["WPL 20", "WPL 25A", "WPL 07HK Premium"],
        cop20.Temperatur,
        ax=ax,
        cbarlabel="COP in % vom Höchswert",
    )

    ax.set_xlabel("Temperatur")
    ax.set_ylabel("Pumpe")
    ax.set_title(
        f"Heatmap der COPs mit {inverter_speed} Inverter-speed bei {vorlauftemperatur[-2:]}°C Vorlauftemperatur"
    )
    annotate_heatmap(
        im, cops, valfmt="{x:.2f}", threshold=4, textcolors=("black", "black")
    )
    fig.tight_layout()
    plt.show()

# Ausgabe von Grafiken

Der erste Regler ist die *Inverter-speed* und der zweite Regler ist die *Vorlauftemperatur*. Die Einstellung dieser Regler verändert die erzeugten Grafiken. Aufgrund der Servergeschwindigkeit werden die Grafiken möglicherweise nicht sofort erstellt, aber ich kann dir versichern, dass sie genau anhand der Excel-Zahlen erstellt werden. Die zweite Grafik enthält mehr Berechnungen, deshalb dauert es etwas länger, bis sie geladen ist als die erste.

## Wie man die Grafiken liest
### Grafik 1 - COP mit x% Inverter-speed bei y°C 
In Grafik 1 ist der COP für jede Pumpe dargestellt. Jede Pumpe wird zur besseren Visualisierung durch ein eigenes Symbol dargestellt. Mit den beiden Reglern kann man die *Inverter-speed* und die *Vorlauftemperatur* einstellen. Man wird auch feststellen, dass für einige der Geschwindigkeiten und Temperaturen der COP fast (wenn nicht identisch) gleich ist. Dies ist **kein** Fehler. Die Daten auf der Website zeigen, dass die Werte tatsächlich gleich sind.

### Grafik 2 - Heatmap der COPs mit x% Inverter-speed bei y°C Vorlauftemperatur
Diese Grafik ist ein wenig schwieriger zu verstehen. Die Daten werden mit Quadraten dargestellt, wobei der genaue COP-Wert innerhalb des Quadrats angegeben ist. Jede Pumpe wird durch eine Zeile dargestellt, und die Kolumnen stellen die Außentemperatur dar. 

Jedes Quadrat hat außerdem einen Farbverlauf, der von dunkelgrün bis dunkelrot reicht. Wenn der COP-Wert am effizientesten ist, wird er als "100 % Effizienz" gekennzeichnet und dunkelgrün eingefärbt. Die anderen Werte sind dann ein Prozentsatz des höchsten Wertes und werden entsprechend eingefärbt.

Zum Beispiel liegt der COP-Wert von Pumpe A bei 4 und der COP-Wert von Pumpe B bei 3. Zwischen diesen beiden Werten ist Pumpe A am effizientesten und wird als 100 % effizient gekennzeichnet. Pumpe B hat dann 75% des Wertes von Pumpe A (4). Daher wird das Quadrat von Pumpe B als 75 % Effizienz eingefärbt, da sie 25 % weniger effizient ist als der Maximalwert. Farblich bedeutet das, dass das Quadrat von Pumpe A dunkelgrün und das Quadrat von Pumpe B dunkelrot eingefärbt wird.

Es kann vorkommen, dass die Pumpen den gleichen Wirkungsgrad haben. Auch das ist **kein** Fehler. Ich habe mir zu viele Haare ausgerissen, um dies zu beheben. Nachdem ich eine Glatze bekommen hatte, stellte ich fest, dass die Werte tatsächlich gleich waren und das Programm richtig war.

In [28]:
@interact(inverter_speed=(0, 100, 5), vorlauftemperatur=(30, 55, 5))
def interact_with_functions(inverter_speed, vorlauftemperatur):
    plotPumps(inverter_speed, vorlauftemperatur)
    plotHeatmaps(inverter_speed, vorlauftemperatur)


interactive(children=(IntSlider(value=50, description='inverter_speed', step=5), IntSlider(value=40, descripti…

Ich hoffe, das hilft!